<a href="https://colab.research.google.com/github/EskelinenElias/ADAML-project1/blob/main/data_modeling_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADAML Project Work: Intermediary Submission 2 - Data Modeling Plan

LUT University \
BM20A6100 Advanced Data Analysis and Machine Learning - Blended teaching, Lpr 1.9.2025-12.12.2025 \
1st Period - Project work - Intermediary Submission 2 - Data modeling plan

Elias Eskelinen, Vili Niemelä & Matti Aalto \
25.9.2025

---

## Intermediary Submission 2 - Instructions

Based on your initial evaluation, the received feedback and the strategies of other groups, perform data pretreatment. There will be a max. 2 pages addition to your project PDF for the pretreatment part.

Breakdown of the points:

0.5p   - feedback incorporation and adjustment of the initial pretreatment plan
0.5p  - data centering and scaling techniques
0.5p  - evaluation of extreme/ missing values and mitigating actions, data synchronization (for time series), sampling
0.5p   - visualizing pretreated data
Modelling plan

Make a modelling plan for your modelling goal. Create a flowchart of the operations needed to accomplish the task, and assign roles to the team members for each step. The start of each flow chart should be the cleaned data, and the end of it should be the modelling goal achieved. If there is more than one modelling goal, the flowchart can take multiple paths. There will be a max. 2 pages addition to your project PDF for the modelling plan part.

0.5p     - modelling goal (What is the purpose of your data analysis?);
0.5p     - model calibration strategy: tools used, methodology, data used;
0.5p     - model validation strategy: scope of validation, methodology;
0.5p     - model testing strategy: metrics and data used;
0.5p     - description of the mathematical methods used in modelling;
0.5p     - model diagram/ operations flowchart.


## Data onboarding